## Regression

Load libraries

In [1]:
import torch
import random
import numpy as np
from sklearn import datasets
from sklearn import model_selection
from typing import Type

Set up session for reproducibility

In [2]:
# Python
random.seed(0)
# Numpy
np.random.seed(1)
# PyTorch
torch.manual_seed(2)
# Ensure that all operations are deterministic on GPU (if used)
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False
# Avoid nondeterministic algorithms
torch.use_deterministic_algorithms(True)

Create a dataset

In [3]:
x, y = datasets.make_regression(
    n_samples=1000,
    n_features=10,
    n_informative=10,
    n_targets=1,
    bias=0.0,
    random_state=3
)

Split the data intro two groups: train and test

In [4]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, random_state=1)
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"--------")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (800, 10)
y_train shape: (800,)
--------
x_test shape: (200, 10)
y_test shape: (200,)


Transform arrays to tensors

In [5]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

Build PyTorch datasets and dataloaders

In [6]:
dataset_train = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
dataset_test  = torch.utils.data.TensorDataset(x_test_tensor,  y_test_tensor)

batch_size = 100
dataloader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
dataloader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=batch_size, shuffle=True)

Define the neural network architecture

In [7]:
class MultilayerPerceptron(torch.nn.Module):
    def __init__(self, input_size: int, layers: list[tuple[int, torch.nn.Module]]) -> None:
        super().__init__()
        # Assert layers is not empty
        assert len(layers) > 0, "layers list cannot be empty"
        # Initialize layers
        self.layers = torch.nn.ModuleList()
        # Build layers
        for output_size, activation in layers:
            self.layers.append(torch.nn.Linear(input_size, output_size))
            input_size = output_size
            if activation is not None:
                assert isinstance(activation, torch.nn.Module), "activation needs to be a subclass of torch.nn.Module"
                self.layers.append(activation)

    def forward(self, x) -> torch.Tensor:
        for layer in self.layers:
            x = layer(x)
        return x

Build a neural network model

In [8]:
model = MultilayerPerceptron(
    input_size=10,
    layers=[
        (12, torch.nn.ReLU()),
        (10, torch.nn.ReLU()),
        ( 5, torch.nn.ReLU()),
        ( 1, None)
    ],
)

Define loss functions for training the neural network

In [9]:
criterion = torch.nn.MSELoss()
criterion

MSELoss()

Define the optimizer for training the neural network

In [10]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

Define function for training the model

In [11]:
ModelT      = Type[torch.nn.Module]
CriterionT  = Type[torch.nn.modules.loss._Loss]
OptimizerT  = Type[torch.optim.Optimizer]
DataLoaderT = Type[torch.utils.data.DataLoader]

def train(model: ModelT, criterion: CriterionT, optimizer: OptimizerT, dataloader: DataLoaderT, num_epochs: int, log_freq: int, log_verbose: bool = False) -> list[float]:
    loss_log = []
    for epoch in range(num_epochs):
        # Initialize loss accumulator
        total_loss = 0.0
        # Train model
        for x_batch, y_batch in dataloader:
            # Forward pass
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Store loss
            total_loss += loss
        
        # Log loss
        if (epoch + 1) % log_freq == 0:
            loss_log.append(total_loss)
            if log_verbose:
                print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {total_loss}")

    return loss_log


Train model

In [12]:
loss_log = train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    dataloader=dataloader_train,
    num_epochs=1000,
    log_freq=100,
    log_verbose=True
)

Epoch [100/1000] | Loss: 4016.767822265625
Epoch [200/1000] | Loss: 1023.3453979492188
Epoch [300/1000] | Loss: 294.7855529785156
Epoch [400/1000] | Loss: 99.35380554199219
Epoch [500/1000] | Loss: 44.753360748291016
Epoch [600/1000] | Loss: 19.249340057373047
Epoch [700/1000] | Loss: 8.172185897827148
Epoch [800/1000] | Loss: 2.9901890754699707
Epoch [900/1000] | Loss: 0.9360741376876831
Epoch [1000/1000] | Loss: 0.28816068172454834


Define function for testing the model

In [13]:
def test(model: ModelT, criterion: CriterionT, dataloader: DataLoaderT) -> float:
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            # Evaluate the model
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            total_loss += loss

    return total_loss

Test the trained model

In [14]:
print(f"Model accuracy on test set: {test(model=model, criterion=criterion, dataloader=dataloader_test)}")

Model accuracy on test set: 0.07015716284513474
